In [1]:
import pandas as pd

df1 = pd.read_csv("Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv")
df2 = pd.read_csv("Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv")
df3 = pd.read_csv("Friday-WorkingHours-Morning.pcap_ISCX.csv")
df4 = pd.read_csv("Monday-WorkingHours.pcap_ISCX.csv")
df5 = pd.read_csv("Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv")
df6 = pd.read_csv("Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv", encoding='latin1')
df7 = pd.read_csv("Tuesday-WorkingHours.pcap_ISCX.csv")
df8 = pd.read_csv("Wednesday-workingHours.pcap_ISCX.csv")



C:\Users\Dani\AppData\Local\Temp\ipykernel_23272\2811516221.py:8: DtypeWarning: Columns (0,1,3,6,84) have mixed types. Specify dtype option on import or set low_memory=False.
  df6 = pd.read_csv("Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv", encoding='latin1')


In [2]:
for i in range(1, 9):
    df = eval(f'df{i}')
    print(f'df{i}.shape: {df.shape}')

df1.shape: (225745, 85)
df2.shape: (286467, 85)
df3.shape: (191033, 85)
df4.shape: (529918, 85)
df5.shape: (288602, 85)
df6.shape: (458968, 85)
df7.shape: (445909, 85)
df8.shape: (692703, 85)


### TRATAMIENTO DE LA COLUMNA TIMESTAMP

In [3]:
df6 = df6.dropna(subset=[" Timestamp"]).copy()

In [4]:
df6[" Timestamp"].isna().sum()
df6.shape

(170366, 85)

limpieza df a df, el df4 se trata diferente.

In [5]:
df4["dt"] = pd.to_datetime(
    df4[" Timestamp"],
    format="%d/%m/%Y %H:%M:%S",
    errors="coerce"
)

print("df4 NaT:", df4["dt"].isna().sum())
print(df4["dt"].min(), df4["dt"].max())

df4 NaT: 0
2017-07-03 01:00:01 2017-07-03 12:59:58


In [6]:
def convert_timestamp_standard(df):
    df["dt"] = pd.to_datetime(
        df[" Timestamp"],
        format="%d/%m/%Y %H:%M",
        errors="coerce"
    )
    return df

In [7]:
df1 = convert_timestamp_standard(df1)
df2 = convert_timestamp_standard(df2)
df3 = convert_timestamp_standard(df3)
#df4 = convert_timestamp_standard(df4)
df5 = convert_timestamp_standard(df5)
df6 = convert_timestamp_standard(df6)
df7 = convert_timestamp_standard(df7)
df8 = convert_timestamp_standard(df8)


In [8]:
for name, df in zip(
    ["df1","df2","df3","df4","df5","df7","df8"],
    [df1,df2,df3,df4,df5,df7,df8]
):
    print(name, "NaT:", df["dt"].isna().sum())


df1 NaT: 0
df2 NaT: 0
df3 NaT: 0
df4 NaT: 0
df5 NaT: 0
df7 NaT: 0
df8 NaT: 0


concatenación

In [9]:
data_list = [df1, df2, df3, df4, df5, df6, df7, df8]

print('Data dimensions: ')
for i, df in enumerate(data_list, start = 1):
  rows, cols = df.shape
  print(f'Data{i} -> {rows} rows, {cols} columns')

Data dimensions: 
Data1 -> 225745 rows, 86 columns
Data2 -> 286467 rows, 86 columns
Data3 -> 191033 rows, 86 columns
Data4 -> 529918 rows, 86 columns
Data5 -> 288602 rows, 86 columns
Data6 -> 170366 rows, 86 columns
Data7 -> 445909 rows, 86 columns
Data8 -> 692703 rows, 86 columns


In [10]:
df = pd.concat(data_list, ignore_index=True)
rows, cols = df.shape

print('New dimension:')
print(f'Number of rows: {rows}')
print(f'Number of columns: {cols}')
print(f'Total cells: {rows * cols}')

New dimension:
Number of rows: 2830743
Number of columns: 86
Total cells: 243443898


In [11]:
col_names = {col: col.strip() for col in df.columns}
df.rename(columns = col_names, inplace = True)

In [12]:
df.columns

Index(['Flow ID', 'Source IP', 'Source Port', 'Destination IP',
       'Destination Port', 'Protocol', 'Timestamp', 'Flow Duration',
       'Total Fwd Packets', 'Total Backward Packets',
       'Total Length of Fwd Packets', 'Total Length of Bwd Packets',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Min Packet Length', 'Max Packet Length', 'Packet Length Mean',
  

In [13]:
df = df.drop(columns=["Timestamp"])
df = df.rename(columns={"dt": "Timestamp"})

In [14]:
df['Timestamp'].dt.day_name().value_counts()

Timestamp
Friday       703245
Wednesday    692703
Monday       529918
Thursday     458968
Tuesday      445909
Name: count, dtype: int64

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2830743 entries, 0 to 2830742
Data columns (total 85 columns):
 #   Column                       Dtype         
---  ------                       -----         
 0   Flow ID                      object        
 1   Source IP                    object        
 2   Source Port                  float64       
 3   Destination IP               object        
 4   Destination Port             float64       
 5   Protocol                     float64       
 6   Flow Duration                float64       
 7   Total Fwd Packets            float64       
 8   Total Backward Packets       float64       
 9   Total Length of Fwd Packets  float64       
 10  Total Length of Bwd Packets  float64       
 11  Fwd Packet Length Max        float64       
 12  Fwd Packet Length Min        float64       
 13  Fwd Packet Length Mean       float64       
 14  Fwd Packet Length Std        float64       
 15  Bwd Packet Length Max        float64       
 16  

In [17]:
pd.options.display.max_columns = 85

df

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Timestamp
0,192.168.10.5-104.16.207.165-54865-443-6,104.16.207.165,443.0,192.168.10.5,54865.0,6.0,3.0,2.0,0.0,12.0,0.0,6.0,6.0,6.0,0.00000,0.0,0.0,0.0,0.0,4.000000e+06,666666.666700,3.0,0.000000,3.0,3.0,3.0,3.00000,0.00000,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0,0.0,666666.666700,0.000000,6.0,6.0,6.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,9.000000,6.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,12.0,0.0,0.0,33.0,-1.0,1.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,2017-07-07 03:30:00
1,192.168.10.5-104.16.28.216-55054-80-6,104.16.28.216,80.0,192.168.10.5,55054.0,6.0,109.0,1.0,1.0,6.0,6.0,6.0,6.0,6.0,0.00000,6.0,6.0,6.0,0.0,1.100917e+05,18348.623850,109.0,0.000000,109.0,109.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,20.0,9174.311927,9174.311927,6.0,6.0,6.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,9.000000,6.0,6.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.0,1.0,6.0,29.0,256.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,2017-07-07 03:30:00
2,192.168.10.5-104.16.28.216-55055-80-6,104.16.28.216,80.0,192.168.10.5,55055.0,6.0,52.0,1.0,1.0,6.0,6.0,6.0,6.0,6.0,0.00000,6.0,6.0,6.0,0.0,2.307692e+05,38461.538460,52.0,0.000000,52.0,52.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,20.0,19230.769230,19230.769230,6.0,6.0,6.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,9.000000,6.0,6.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.0,1.0,6.0,29.0,256.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,2017-07-07 03:30:00
3,192.168.10.16-104.17.241.25-46236-443-6,104.17.241.25,443.0,192.168.10.16,46236.0,6.0,34.0,1.0,1.0,6.0,6.0,6.0,6.0,6.0,0.00000,6.0,6.0,6.0,0.0,3.529412e+05,58823.529410,34.0,0.000000,34.0,34.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,20.0,29411.764710,29411.764710,6.0,6.0,6.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,9.000000,6.0,6.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.0,1.0,6.0,31.0,329.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,2017-07-07 03:30:00
4,192.168.10.5-104.19.196.102-54863-443-6,104.19.196.102,443.0,192.168.10.5,54863.0,6.0,3.0,2.0,0.0,12.0,0.0,6.0,6.0,6.0,0.00000,0.0,0.0,0.0,0.0,4.000000e+06,666666.666700,3.0,0.000000,3.0,3.0,3.0,3.00000,0.00000,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0,0.0,666666.666700,0.000000,6.0,6.0,6.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,9.000000,6.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,12.0,0.0,0.0,32.0,-1.0,1.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,2017-07-07 03:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [18]:
dups = df[df.duplicated()]
len(dups)

203

In [19]:
df.drop_duplicates(inplace = True)


In [20]:
missing = df.isna().sum()
print(missing[missing > 0].to_string())

Flow Bytes/s    1357


In [22]:
# Ver cuántos NaN tiene cada fila
fila_nan_count = df.isna().sum(axis=1)

# Filas que tienen muchos NaNs
print(fila_nan_count.sort_values(ascending=False).head())

2277204    1
2329145    1
1878352    1
2356742    1
2329155    1
dtype: int64


In [23]:
df.loc[1692131].to_string()

'Flow ID                        192.168.10.3-192.168.10.5-88-49182-6\nSource IP                                              192.168.10.5\nSource Port                                                 49182.0\nDestination IP                                         192.168.10.3\nDestination Port                                               88.0\nProtocol                                                        6.0\nFlow Duration                                                 640.0\nTotal Fwd Packets                                               7.0\nTotal Backward Packets                                          4.0\nTotal Length of Fwd Packets                                   440.0\nTotal Length of Bwd Packets                                   358.0\nFwd Packet Length Max                                         220.0\nFwd Packet Length Min                                           0.0\nFwd Packet Length Mean                                    62.857143\nFwd Packet Length Std            

In [24]:
df = df.drop(index=1692131)

Valores Infinitos

In [ ]:
import numpy as np

numeric_cols = df.select_dtypes(include = np.number).columns
inf_count = np.isinf(df[numeric_cols]).sum()
print(inf_count[inf_count > 0])

Flow Bytes/s      1506
Flow Packets/s    2863
dtype: int64


Reemplazo por NaN

In [ ]:

print(f'Nan Actuales {df.isna().sum().sum()}')

df.replace([np.inf, -np.inf], np.nan, inplace = True)

print(f'Nan Totales: {df.isna().sum().sum()}')

Initial missing values: 1357
Missing values after processing infinite values: 5726


In [27]:
missing = df.isna().sum()
print(missing.loc[missing > 0])

Flow Bytes/s      2863
Flow Packets/s    2863
dtype: int64


In [28]:
#pct de valores nan con respecto al total
print(f" {round(2863 * 100/df.shape[0],2)} %")


 0.1 %


In [29]:
# Ver cuántos NaN tiene cada fila
fila_nan_count = df.isna().sum(axis=1)
print(fila_nan_count.sort_values(ascending=False).head())


399271     2
2112840    2
2485598    2
452934     2
807423     2
dtype: int64


In [ ]:
#df[['Flow Bytes/s', 'Flow Packets/s']].head(20)
listaFlowBytes = df['Flow Bytes/s'].head(300)

for i in listaFlowBytes:
  print(i)


imputar flowBytes y flowPackets a 0, ya que es el resultado de dividir total Bytes / Flow Duration, algunos casos tendrá inf/nan

In [31]:
df['Flow Bytes/s'].fillna(0, inplace=True)
df['Flow Packets/s'].fillna(0, inplace=True)

C:\Users\Dani\AppData\Local\Temp\ipykernel_23272\3283618173.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Flow Bytes/s'].fillna(0, inplace=True)
C:\Users\Dani\AppData\Local\Temp\ipykernel_23272\3283618173.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, wh

In [32]:
df[["Fwd Header Length", "Fwd Header Length.1"]].head(20)
df["Fwd Header Length"].equals(df["Fwd Header Length.1"])

True

In [33]:
df.drop(columns=["Fwd Header Length.1"], inplace=True)

In [34]:
df.shape

(2830539, 84)

Columnas sobrantes

In [35]:
df[[
    "Fwd Avg Bytes/Bulk", "Fwd Avg Packets/Bulk", "Fwd Avg Bulk Rate",
    "Bwd Avg Bytes/Bulk", "Bwd Avg Packets/Bulk", "Bwd Avg Bulk Rate"
]].describe()

,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate
count,2830539.0,2830539.0,2830539.0,2830539.0,2830539.0,2830539.0
mean,0.0,0.0,0.0,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0
min,0.0,0.0,0.0,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0,0.0,0.0
50%,0.0,0.0,0.0,0.0,0.0,0.0
75%,0.0,0.0,0.0,0.0,0.0,0.0
max,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
cols_varianza_cero = df.columns[df.nunique() <= 1]
for i in cols_varianza_cero:
    print(i)
    
for col in cols_varianza_cero:
    print(f" Descripción de la columna: '{col}'")
    print(df[col].describe())

Bwd PSH Flags
Bwd URG Flags
Fwd Avg Bytes/Bulk
Fwd Avg Packets/Bulk
Fwd Avg Bulk Rate
Bwd Avg Bytes/Bulk
Bwd Avg Packets/Bulk
Bwd Avg Bulk Rate
 Descripción de la columna: 'Bwd PSH Flags'
count    2830539.0
mean           0.0
std            0.0
min            0.0
25%            0.0
50%            0.0
75%            0.0
max            0.0
Name: Bwd PSH Flags, dtype: float64
 Descripción de la columna: 'Bwd URG Flags'
count    2830539.0
mean           0.0
std            0.0
min            0.0
25%            0.0
50%            0.0
75%            0.0
max            0.0
Name: Bwd URG Flags, dtype: float64
 Descripción de la columna: 'Fwd Avg Bytes/Bulk'
count    2830539.0
mean           0.0
std            0.0
min            0.0
25%            0.0
50%            0.0
75%            0.0
max            0.0
Name: Fwd Avg Bytes/Bulk, dtype: float64
 Descripción de la columna: 'Fwd Avg Packets/Bulk'
count    2830539.0
mean           0.0
std            0.0
min            0.0
25%            0.0
50% 

In [37]:
columnas_a_quitar = [
    'Bwd PSH Flags', 'Bwd URG Flags',
    'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk', 'Fwd Avg Bulk Rate',
    'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate'
]

df.drop(columns=columnas_a_quitar, inplace=True)

In [38]:
df.shape

(2830539, 76)

In [39]:
pd.set_option('display.max_rows', None)
print(df.dtypes)

Flow ID                                object
Source IP                              object
Source Port                           float64
Destination IP                         object
Destination Port                      float64
Protocol                              float64
Flow Duration                         float64
Total Fwd Packets                     float64
Total Backward Packets                float64
Total Length of Fwd Packets           float64
Total Length of Bwd Packets           float64
Fwd Packet Length Max                 float64
Fwd Packet Length Min                 float64
Fwd Packet Length Mean                float64
Fwd Packet Length Std                 float64
Bwd Packet Length Max                 float64
Bwd Packet Length Min                 float64
Bwd Packet Length Mean                float64
Bwd Packet Length Std                 float64
Flow Bytes/s                          float64
Flow Packets/s                        float64
Flow IAT Mean                     

### EXTRAER TIMESTAMP PARA POSTERIOR USO EN VENTANAS TEMPORALES

se creará un parquet de un dataframe de una sola columna llamada Timestamp

In [40]:
dfOnlyTimestamp = df[['Timestamp']]

In [41]:
dfOnlyTimestamp.shape

(2830539, 1)

Parquet de df de Timestamp guardado

In [43]:
dfOnlyTimestamp.to_parquet("Timestamp_Datetime_Terminado\\Timestamp_Tipo_Datetime.parquet")

ELIMINAR CUALQUIER COLUMNA STR PARA PCA


In [45]:
cols_drop = [
    "Flow ID",
    "Source IP",
    "Destination IP",
    "Source Port",
    "Destination Port",
    "Protocol",
    "Timestamp",              
    "Fwd Header Length.1",
    "Fwd Avg Bytes/Bulk",
    "Fwd Avg Packets/Bulk",
    "Fwd Avg Bulk Rate",
    "Bwd Avg Bytes/Bulk",
    "Bwd Avg Packets/Bulk",
    "Bwd Avg Bulk Rate"
]

df.drop(columns=cols_drop, inplace=True, errors="ignore")

In [46]:
df.shape

(2830539, 69)

### ESTA DISPONIBLE PARA USAR EL CAMBIO DE INT64 A INT32 POR OPTIMIZAR:

In [ ]:
old_memory_usage = df.memory_usage().sum() / 1024 ** 2
print(f'Initial memory usage: {old_memory_usage:.2f} MB')
for col in df.columns:
    col_type = df[col].dtype
    if col_type != object:
        c_min = df[col].min()
        c_max = df[col].max()
        #Cambio de float64 a float32
        if str(col_type).find('float') >= 0 and c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
            df[col] = df[col].astype(np.float32)

        #Cambio de int64 a int32
        elif str(col_type).find('int') >= 0 and c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
            df[col] = df[col].astype(np.int32)

new_memory_usage = df.memory_usage().sum() / 1024 ** 2
print(f"Final memory usage: {new_memory_usage:.2f} MB")



Initial memory usage: 1511.67 MB
Final memory usage: 777.43 MB


In [49]:
print(df.dtypes.to_string())

Flow Duration                  float32
Total Fwd Packets              float32
Total Backward Packets         float32
Total Length of Fwd Packets    float32
Total Length of Bwd Packets    float32
Fwd Packet Length Max          float32
Fwd Packet Length Min          float32
Fwd Packet Length Mean         float32
Fwd Packet Length Std          float32
Bwd Packet Length Max          float32
Bwd Packet Length Min          float32
Bwd Packet Length Mean         float32
Bwd Packet Length Std          float32
Flow Bytes/s                   float32
Flow Packets/s                 float32
Flow IAT Mean                  float32
Flow IAT Std                   float32
Flow IAT Max                   float32
Flow IAT Min                   float32
Fwd IAT Total                  float32
Fwd IAT Mean                   float32
Fwd IAT Std                    float32
Fwd IAT Max                    float32
Fwd IAT Min                    float32
Bwd IAT Total                  float32
Bwd IAT Mean             

Instalación y creación del archivo Parquet

In [47]:
!pip install pyarrow


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
!pip install fastparquet

In [52]:
df.to_parquet("Limpieza_Completada\\cicids2017_CleanComplete.parquet")